### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import math
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

### Convert & Create Dataframes

In [ ]:
# Read in .tsv files
contacts = pd.read_csv('/work/contacts.tsv', sep='\t')
searches = pd.read_csv('/work/searches.tsv', sep='\t')

contacts['ds_checkin'] = pd.to_datetime(contacts['ds_checkin'])
contacts['ds_checkout'] = pd.to_datetime(contacts['ds_checkout'])
contacts['ts_contact_at'] = pd.to_datetime(contacts['ts_contact_at'])
contacts['ts_reply_at'] = pd.to_datetime(contacts['ts_reply_at'])
contacts['ts_accepted_at'] = pd.to_datetime(contacts['ts_accepted_at'])
contacts['ts_booking_at'] = pd.to_datetime(contacts['ts_booking_at'])
contacts['ds_length'] = (contacts['ds_checkout'] - contacts['ds_checkin']).dt.days

searches = searches.drop(columns='filter_neighborhoods')
searches['ds_checkin'] = pd.to_datetime(searches['ds_checkin'])
searches['ds_checkout'] = pd.to_datetime(searches['ds_checkout'])
searches['ds_length'] = (searches['ds_checkout'] - searches['ds_checkin']).dt.days
searches['ds'] = pd.to_datetime(searches['ds'])
searches = searches.dropna(subset=['ds'])
searches['advance_search_days'] = (searches['ds_checkin'] - searches['ds']).dt.days
searches = searches.dropna(subset=['ds_checkin'])

### Combine Contacts and Searches Dataframe

In [ ]:
combined_df = _deepnote_execute_sql('SELECT\n    s.*,\n    c.ts_booking_at\nFROM\n    searches s\nLEFT JOIN (\n    SELECT id_guest, ds_checkin, ds_checkout, MAX(ts_booking_at) AS ts_booking_at\n    FROM contacts\n    GROUP BY id_guest, ds_checkin, ds_checkout\n) c ON s.id_user = c.id_guest AND s.ds_checkin = c.ds_checkin AND s.ds_checkout = c.ds_checkout;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
combined_df

,ds,id_user,ds_checkin,ds_checkout,n_searches,n_nights,n_guests_min,n_guests_max,origin_country,filter_price_min,filter_price_max,filter_room_types,ds_length,advance_search_days,ts_booking_at
0,2014-10-01,00fb5de4-9865-420f-bc85-22521121a1ca,2014-10-03,2014-10-05,15,2.0,2,2,DE,NaN,NaN,None,2.0,2.0,NaT
1,2014-10-01,0187b377-ad1c-4da4-8a17-568b2c9098f4,2015-03-20,2015-03-24,7,4.0,1,1,CA,NaN,NaN,None,4.0,170.0,NaT
2,2014-10-01,024493fb-a47b-49d2-9844-cc8ba21d6e02,2014-10-04,2014-10-06,5,2.0,2,2,CA,NaN,NaN,None,2.0,3.0,2014-10-02 07:54:35
3,2014-10-01,0246d4da-9008-4310-92ef-34f2e8b9ee13,2014-10-16,2014-10-21,18,5.0,1,1,GB,0.0,9.100000e+01,Private room,5.0,15.0,2014-10-07 21:31:42
4,2014-10-01,034acf8b-056e-483c-9064-735b064bc22a,2014-10-18,2014-10-21,7,3.0,2,2,GB,0.0,3.260000e+02,None,3.0,17.0,2014-10-03 13:09:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23883,2014-10-14,fe25dc1d-3290-48fe-a052-b384c6e3c3b3,2014-10-31,2014-11-01,14,1.0,1,2,US,35.0,1.073742e+09,",Entire home/apt,Private room,Entire home/apt",1.0,17.0,NaT
23884,2014-10-14,fe7aeb03-e0dc-4dd9-b385-9fc3ba4d0e11,2014-11-04,2014-11-07,2,3.0,1,4,IE,NaN,NaN,Entire home/apt,3.0,21.0,NaT
23885,2014-10-14,ff06b374-0e81-4f98-a81c-d3a599b5c224,2014-11-17,2014-11-30,2,13.0,2,2,IT,NaN,NaN,",Private room",13.0,34.0,NaT
23886,2014-10-14,ff2cd5cb-7c03-4404-9dd3-2201940da966,2014-11-11,2014-11-15,33,4.0,1,1,FR,NaN,NaN,None,4.0,28.0,NaT


### Model

In [ ]:
#predict if user is able to get a booking or not
#predictors: how soon u need it and date range

#copying predictors over to new dataframe
df = combined_df[['advance_search_days', 'ds_length']]
#prepare target variable
df['is_booking_successful'] = ~combined_df['ts_booking_at'].isnull()

#creating column to classify uniqueness of booking date range
#date_range_frequency = df['ds_length'].value_counts().to_dict()

#higher the number the more unique
# max_frequency = max(date_range_frequency.values())
# df['date_range_uniqueness'] = df['ds_length'].apply(lambda x: max_frequency - date_range_frequency[x] + 1)
df

/tmp/ipykernel_1169/348201118.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_booking_successful'] = ~combined_df['ts_booking_at'].isnull()


,advance_search_days,ds_length,is_booking_successful
0,2.0,2.0,False
1,170.0,4.0,False
2,3.0,2.0,True
3,15.0,5.0,True
4,17.0,3.0,True
...,...,...,...
23883,17.0,1.0,False
23884,21.0,3.0,False
23885,34.0,13.0,False
23886,28.0,4.0,False


In [ ]:
#undersampling all the unsuccessful bookings to deal with class imbalance
df_majority = df[df['is_booking_successful'] == False]
df_minority = df[df['is_booking_successful'] == True]

df_majority_undersampled = df_majority.sample(len(df_minority), random_state=42)

df_undersampled = pd.concat([df_majority_undersampled, df_minority])

#train the logistic regression model on the scaled training data and evaluate results
X = df_undersampled[['advance_search_days', 'ds_length']]
y = df_undersampled['is_booking_successful']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score:", accuracy)

              precision    recall  f1-score   support

       False       0.67      0.36      0.47      1046
        True       0.55      0.81      0.65       989

    accuracy                           0.58      2035
   macro avg       0.61      0.59      0.56      2035
weighted avg       0.61      0.58      0.56      2035

Accuracy score: 0.5803439803439804


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b437c3b3-3aa6-4b0e-b468-e8e7345d65e9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>